In [ ]:
from common import *

use_hvplot()

In [ ]:
client = start_cluster(5, 12)
client_ip_and_port(client)

# SWE output comparison between iSnobal and Snow-17

## CBRFC

In [ ]:
CBRFC_CSV = pd.read_csv(
    CBRFC_DIR / 'SNOW17ALEC2_2018to2021.csv',
    parse_dates=True,
    index_col=0,
    header=0,
    names=['Lower', 'Middle', 'Upper', 'Average'],
    dtype={'Lower': np.float64, 'Middle': np.float64, 'Upper': np.float64},
)
CBRFC_CSV.Lower *= INCH_TO_MM
CBRFC_CSV.Middle *= INCH_TO_MM
CBRFC_CSV.Upper *= INCH_TO_MM

## iSnobal 

In [ ]:
wy_swe = xr.open_mfdataset(
    (SNOBAL_DIR / 'wy20*' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

wy_swe.coords['mask'] = (('y', 'x'), cbrfc_zones())

In [ ]:
hlf = wy_swe.where(wy_swe.mask == ALEC2HLF).specific_mass.mean(("x", "y")).compute()
hmf = wy_swe.where(wy_swe.mask == ALEC2HMF).specific_mass.mean(("x", "y")).compute()
huf = wy_swe.where(wy_swe.mask == ALEC2HUF).specific_mass.mean(("x", "y")).compute()

## Compare CBRFC to iSnobal

In [ ]:
(
    huf.hvplot(label='Upper (iSnobal)', color='indigo') * \
    CBRFC_CSV.Upper.plot(color='mediumorchid', label='Upper (Snow-17)', line_dash='dashed') * \
    hmf.hvplot(label='Middle (iSnobal)', color='teal') * \
    CBRFC_CSV.Middle.plot(color='cadetblue', label='Middle (Snow-17)', line_dash='dashed') * \
    hlf.hvplot(label='Lower (iSnobal)', color='gold') * \
    CBRFC_CSV.Lower.plot(color='orange', label='Lower (Snow-17)', line_dash='dashed')
).opts(
    title='iSnobal vs Snow-17 SWE', xlabel='Date',
    ylabel='SWE (mm)', yformatter='%d',
    width=1080, height=580,
    fontsize={'title': 16, 'labels': 14, 'xticks': 14, 'yticks': 14, 'legend': 14}
)

In [ ]:
huf.hvplot(label='Upper', color='indigo').opts(
    title='iSnobal SWE', xlabel='Date',
    ylabel='SWE (mm)', yformatter='%d',
    width=1280, height=640,
    **BOKEH_FONT,
) * \
hmf.hvplot(label='Middle', color='teal') * \
hlf.hvplot(label='Lower', color='gold') 

In [ ]:
CBRFC_CSV.Upper.plot(color='indigo', line_dash='dashed').opts(
    title='Snow-17 SWE', xlabel='Date', ylabel='SWE (mm)', 
    width=1280, height=640, **BOKEH_FONT,
) * \
CBRFC_CSV.Middle.plot(color='teal', line_dash='dashed') * \
CBRFC_CSV.Lower.plot(color='gold', line_dash='dashed')

In [ ]:
client.shutdown()